In [ ]:
import csv
import os
import pandas as pd
import altair as alt
from datetime import datetime, timedelta
import numpy as np

In [ ]:
# only to read from csv

df_3wfl = pd.read_csv('data/3wboundaries-weather-dst-adj-bins-80days.zip')

df_3wfl
df_3wfl['b_ts'] = pd.to_datetime(df_3wfl['b_ts'], format='%Y-%m-%d %H:%M:%S')
df_3wfl['pickup_datetime_1min'] = pd.to_datetime(df_3wfl['pickup_datetime_1min'], format='%Y-%m-%d %H:%M:%S')
df_3wfl['pickup_datetime_same_time'] = pd.to_datetime(df_3wfl['pickup_datetime_same_time'], format='%Y-%m-%d %H:%M:%S')
df_3wfl.dtypes

b_id                                  int64
b_ts                         datetime64[ns]
pickup_datetime_1min         datetime64[ns]
dist_bin                            float64
rides_per_minute                      int64
which_week                           object
sequence                              int64
pickup_datetime_same_time    datetime64[ns]
dtype: object

In [ ]:
x = df_3wfl['which_week']
condlist = [x=='Current week', x!='Current week']
choicelist = [1, 0]
df_3wfl['which_week_dummy'] = np.select(condlist, choicelist)

In [ ]:
def make_before_after_rain_dummy(df_gb):         
    
    # here a series is being passed to the comp_dates function through 'apply'
    df_gb['before_after_rain_dummy'] = df_gb.apply(
        lambda row: 0 if row['pickup_datetime_1min'] <= row['b_ts'] else 1, axis=1
    )   
    return df_gb

df_3wfl = df_3wfl.groupby('b_id').apply(make_before_after_rain_dummy)

In [ ]:
df_3wfl.shape

(1455509, 10)

In [ ]:
def make_within_two_hours_dummy(df_gb):         
    
    # here a series is being passed to the comp_dates function through 'apply'
    df_gb['within_two_hours_dummy'] = df_gb.apply(
        lambda row: 1 if row['b_ts'] + timedelta(hours=-2) <= row['pickup_datetime_same_time'] <= row['b_ts']  + timedelta(hours=2) else 0, axis=1
    )   
    return df_gb

df_3wfl = df_3wfl.groupby('b_id').apply(make_within_two_hours_dummy)



In [ ]:
df_3wfl.head()

,b_id,b_ts,pickup_datetime_1min,dist_bin,rides_per_minute,which_week,sequence,pickup_datetime_same_time,which_week_dummy,before_after_rain_dummy,within_two_hours_dummy
0,0,2010-01-17 11:51:00,2010-01-10 05:51:00,8.0,10,Week before,1,2010-01-17 05:51:00,0,0,0
1,0,2010-01-17 11:51:00,2010-01-10 05:51:00,16.0,3,Week before,2,2010-01-17 05:51:00,0,0,0
2,0,2010-01-17 11:51:00,2010-01-10 05:51:00,2.0,12,Week before,3,2010-01-17 05:51:00,0,0,0
3,0,2010-01-17 11:51:00,2010-01-10 05:51:00,1.0,6,Week before,4,2010-01-17 05:51:00,0,0,0
4,0,2010-01-17 11:51:00,2010-01-10 05:52:00,2.0,8,Week before,5,2010-01-17 05:52:00,0,0,0


In [ ]:
df_3wfl2hr = df_3wfl[df_3wfl['within_two_hours_dummy']==1]

In [ ]:
df_3wfl2hr[df_3wfl2hr['b_ts'] == datetime(2010, 2, 23, 6, 51, 0)]

,b_id,b_ts,pickup_datetime_1min,dist_bin,rides_per_minute,which_week,sequence,pickup_datetime_same_time,which_week_dummy,before_after_rain_dummy,within_two_hours_dummy
91425,5,2010-02-23 06:51:00,2010-02-16 04:51:00,2.00,5,Week before,1752,2010-02-23 04:51:00,0,0,1
91426,5,2010-02-23 06:51:00,2010-02-16 04:51:00,4.00,7,Week before,1753,2010-02-23 04:51:00,0,0,1
91427,5,2010-02-23 06:51:00,2010-02-16 04:51:00,16.00,5,Week before,1754,2010-02-23 04:51:00,0,0,1
91428,5,2010-02-23 06:51:00,2010-02-16 04:51:00,1.00,3,Week before,1755,2010-02-23 04:51:00,0,0,1
91429,5,2010-02-23 06:51:00,2010-02-16 04:51:00,8.00,9,Week before,1756,2010-02-23 04:51:00,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
104849,5,2010-02-23 06:51:00,2010-03-02 08:51:00,8.00,17,Week after,3396,2010-02-23 08:51:00,0,1,1
104850,5,2010-02-23 06:51:00,2010-03-02 08:51:00,16.00,7,Week after,3397,2010-02-23 08:51:00,0,1,1
104851,5,2010-02-23 06:51:00,2010-03-02 08:51:00,0.50,14,Week after,3398,2010-02-23 08:51:00,0,1,1
104852,5,2010-02-23 06:51:00,2010-03-02 08:51:00,32.00,3,Week after,3399,2010-02-23 08:51:00,0,1,1


In [ ]:
a= '''0
2
3
5
16
19
39
40
43
46
52
60

'''


b = '''7
8
13
21
22
24
25
34
37
41
45
'''

great = list(map(int, a.split())) # CB list
good = list(map(int, b.split())) # CB list
unusual = [9,54] # CB list



In [ ]:
# works
df_3w_sum = df_3wfl.groupby(by=['b_id', 'b_ts', 'pickup_datetime_same_time', 'pickup_datetime_1min', 'which_week' ]).sum('rides_per_minute').reset_index()

In [ ]:
#df_3wfl2hr['rides_per_block'] = df_3wfl2hr.groupby(by=['b_id', 'which_week_dummy', 'before_after_rain_dummy' ])['rides_per_minute'].transform('sum')

In [ ]:
df_3w_sum

,b_id,b_ts,pickup_datetime_same_time,pickup_datetime_1min,which_week,dist_bin,rides_per_minute,sequence,which_week_dummy,before_after_rain_dummy,within_two_hours_dummy
0,0,2010-01-17 11:51:00,2010-01-17 05:51:00,2010-01-10 05:51:00,Week before,27.00,31,10,0,0,0
1,0,2010-01-17 11:51:00,2010-01-17 05:51:00,2010-01-17 05:51:00,Current week,59.50,40,28,7,0,0
2,0,2010-01-17 11:51:00,2010-01-17 05:51:00,2010-01-24 05:51:00,Week after,31.25,49,28,0,7,0
3,0,2010-01-17 11:51:00,2010-01-17 05:52:00,2010-01-10 05:52:00,Week before,63.00,55,56,0,0,0
4,0,2010-01-17 11:51:00,2010-01-17 05:52:00,2010-01-17 05:52:00,Current week,63.50,73,92,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...
172314,79,2010-12-26 09:51:00,2010-12-26 15:49:00,2010-12-26 15:49:00,Current week,63.75,223,46404,8,8,0
172315,79,2010-12-26 09:51:00,2010-12-26 15:50:00,2010-12-19 15:50:00,Week before,63.75,388,49148,0,0,0
172316,79,2010-12-26 09:51:00,2010-12-26 15:50:00,2010-12-26 15:50:00,Current week,31.75,232,46468,8,8,0
172317,79,2010-12-26 09:51:00,2010-12-26 15:51:00,2010-12-19 15:51:00,Week before,31.75,89,43057,0,0,0


In [ ]:



#for b in df_3w_sum['b_ts'].unique()[0:2]:
   
    
# https://stackoverflow.com/questions/13703720/converting-between-datetime-timestamp-and-datetime64
b = pd.Timestamp(df_3w_sum['b_ts'].unique()[0])


df_single_12hrs = df_3w_sum[df_3w_sum['b_ts'] == b]

df_single_12hrs.to_csv('data/single20100117.csv', index=False)


line = alt.Chart(pd.DataFrame({'x': [str(b)]})).mark_rule().encode(x=alt.X('x:T', title='') )    

# https://altair-viz.github.io/user_guide/times_and_dates.html
rides = alt.Chart(df_single_12hrs, title='B_ID: ' + str(df_single_12hrs.iloc[0]['b_id']) + " - Rides around " + str(b.strftime('%A')) + ' - ' + str(b) + '(12 hour window)').mark_line().encode(
    #x=alt.X('hoursminutes(pickup_datetime_1min):Q', title='Date/time', 
    #x=alt.X('utchoursminutes(pickup_datetime_1min_str):T', title='Date/time', 
    x=alt.X('pickup_datetime_same_time:T', title='Date/time', 
            axis=alt.Axis(
                        tickCount=18,
                    )
            ), 
    #x=alt.X('sequence:Q', axis=None, title='Date/time' ),     
    y=alt.Y('rides_per_minute:Q', title='Rides per minute'),
    color=alt.Color('which_week:N', title='Which week', scale=alt.Scale(
                                            domain=['Week before', 'Current week', 'Week after'],
                                            range=['#DB9EA6 ', '#FFE303', '#adc0d8']))
).properties(
width=700,
    height=250
).interactive()


(rides+line).display()


alt.LayerChart(...)

In [ ]:
# df_3wfl = df_3w_sum

# # https://stackoverflow.com/questions/13703720/converting-between-datetime-timestamp-and-datetime64
# b = pd.Timestamp(datetime(2010, 1, 17, 11, 51, 0))

# df_single = df_3wfl[df_3wfl['b_ts'] == b]

# make two hour windows before and after boundary
ts_before = b + timedelta(hours=-2)
ts_after = b + timedelta(hours=2)

# get all three weeks in plus/minus 2 hour window
df_sing_3w_4hrs = df_single_12hrs[(df_single_12hrs['pickup_datetime_same_time']>= ts_before) & (df_single_12hrs['pickup_datetime_same_time'] <= ts_after)] 
df_sing_3w_4hrs


,b_id,b_ts,pickup_datetime_same_time,pickup_datetime_1min,which_week,dist_bin,rides_per_minute,sequence,which_week_dummy,before_after_rain_dummy,within_two_hours_dummy
720,0,2010-01-17 11:51:00,2010-01-17 09:51:00,2010-01-10 09:51:00,Week before,63.75,233,17937,0,0,9
721,0,2010-01-17 11:51:00,2010-01-17 09:51:00,2010-01-17 09:51:00,Current week,31.75,228,15524,8,0,8
722,0,2010-01-17 11:51:00,2010-01-17 09:51:00,2010-01-24 09:51:00,Week after,31.75,243,15828,0,8,8
723,0,2010-01-17 11:51:00,2010-01-17 09:52:00,2010-01-10 09:52:00,Week before,63.75,255,18018,0,0,9
724,0,2010-01-17 11:51:00,2010-01-17 09:52:00,2010-01-17 09:52:00,Current week,63.75,246,15588,8,0,8
...,...,...,...,...,...,...,...,...,...,...,...
1438,0,2010-01-17 11:51:00,2010-01-17 13:50:00,2010-01-17 13:50:00,Current week,63.75,488,35883,9,9,9
1439,0,2010-01-17 11:51:00,2010-01-17 13:50:00,2010-01-24 13:50:00,Week after,63.75,512,36198,0,9,9
1440,0,2010-01-17 11:51:00,2010-01-17 13:51:00,2010-01-10 13:51:00,Week before,63.75,377,36675,0,0,9
1441,0,2010-01-17 11:51:00,2010-01-17 13:51:00,2010-01-17 13:51:00,Current week,63.75,509,35964,9,9,9


In [ ]:
# current week
df_sing_3w_cw = df_sing_3w_4hrs[df_sing_3w_4hrs['which_week']=='Current week']
# other weeks
df_sing_3w_ow = df_sing_3w_4hrs[df_sing_3w_4hrs['which_week']!='Current week']


lstDictsBID = []

In [ ]:
treatment_pre = df_sing_3w_cw[df_sing_3w_cw['pickup_datetime_same_time']< b]['rides_per_minute'].mean()
# https://stackoverflow.com/questions/2853683/what-is-the-preferred-syntax-for-initializing-a-dict-curly-brace-literals-or
lstDictsBID.append({'b_id': 0, 'b':b, 'which': 'treatment', 'timestamp': b + timedelta(hours=-1), 'rides_per_min': treatment_pre})

treatment_post = df_sing_3w_cw[df_sing_3w_cw['pickup_datetime_same_time']>= b]['rides_per_minute'].mean()
lstDictsBID.append({'b_id': 0, 'b':b, 'which': 'treatment', 'timestamp': b + timedelta(hours=1), 'rides_per_min': treatment_post})


lstDictsBID.append({'b_id': 0, 'b':b, 'which': 'counter-factual', 'timestamp': b + timedelta(hours=-1), 'rides_per_min': treatment_pre})


In [ ]:
control_ow_pre = df_sing_3w_ow[df_sing_3w_ow['pickup_datetime_same_time']< b]['rides_per_minute'].mean()
lstDictsBID.append({'b_id': 0, 'b':b, 'which': 'control', 'timestamp': b + timedelta(hours=-1), 'rides_per_min': control_ow_pre})
control_ow_post = df_sing_3w_ow[df_sing_3w_ow['pickup_datetime_same_time']>= b]['rides_per_minute'].mean()
lstDictsBID.append({'b_id': 0, 'b':b, 'which': 'control', 'timestamp': b + timedelta(hours=1), 'rides_per_min': control_ow_post})

treat_counter_fact = control_ow_post - control_ow_pre + treatment_pre
lstDictsBID.append({'b_id': 0, 'b':b, 'which': 'counter-factual', 'timestamp': b + timedelta(hours=1), 'rides_per_min': treat_counter_fact})


In [ ]:
# lstDictsBID.append({'b_id': 0, 'b':b, 'which': 'control', 'timestamp': b + timedelta(hours=-1), 'rides_per_min': control_pre})


df_did = pd.DataFrame(lstDictsBID)
df_did

,b_id,b,which,timestamp,rides_per_min
0,0,2010-01-17 11:51:00,treatment,2010-01-17 10:51:00,279.625000
1,0,2010-01-17 11:51:00,treatment,2010-01-17 12:51:00,470.041322
2,0,2010-01-17 11:51:00,counter-factual,2010-01-17 10:51:00,279.625000
3,0,2010-01-17 11:51:00,control,2010-01-17 10:51:00,311.520833
4,0,2010-01-17 11:51:00,control,2010-01-17 12:51:00,415.351240
5,0,2010-01-17 11:51:00,counter-factual,2010-01-17 12:51:00,383.455406


In [ ]:
boundary_line = alt.Chart(pd.DataFrame({'x': [str(b)]})).mark_rule().encode(x=alt.X('x:T', title='') ) 

# https://altair-viz.github.io/user_guide/times_and_dates.html
rides = alt.Chart(df_sing_3w_4hrs, title='B_ID: ' + str(df_single_12hrs.iloc[0]['b_id']) + " - Rides around " + str(b.strftime('%A')) + ' - ' + str(b)).mark_line().encode(
    #x=alt.X('hoursminutes(pickup_datetime_1min):Q', title='Date/time', 
    #x=alt.X('utchoursminutes(pickup_datetime_1min_str):T', title='Date/time', 
    x=alt.X('pickup_datetime_same_time:T', title='Date/time', 
            axis=alt.Axis(
                        tickCount=18,
                    )
            ),     
    y=alt.Y('rides_per_minute:Q', title='Rides per minute', scale=alt.Scale(domain=[0, 700])),
    color=alt.Color('which_week:N', title='Which week', scale=alt.Scale(
                                            domain=['Week before', 'Current week', 'Week after'],
                                            range=['#DB9EA6 ', '#FFE303', '#adc0d8']))
).properties(
width=700,
    height=250
)

did_points = alt.Chart(df_did).mark_circle(size=100).encode(
                    x=alt.X('timestamp:T', title=''),
                    y=alt.Y('rides_per_min:Q', title=''),
                    color=alt.Color('which:N', title='Rides per minute means',
                     scale=alt.Scale(domain=['control', 'treatment', 'counter-factual'], 
                     range=['red','blue', 'blue']),
                     legend=None),                                     
                    )



did_lines = alt.Chart(df_did).mark_line().encode(
                    x=alt.X('timestamp:T', title=''),
                    y=alt.Y('rides_per_min:Q', title=''),
                    color=alt.Color('which:N', title='Rides per minute means', scale=alt.Scale(domain=['control', 'treatment'], range=['red', 'blue'])),                                     
                    )

count_fact_line = alt.Chart(df_did[df_did['which'] == 'counter-factual']).mark_line(strokeWidth=1).encode(
                    x=alt.X('timestamp:T', title=''),
                    y=alt.Y('rides_per_min:Q', title=''),
                    color=alt.Color('which:N', scale=alt.Scale(domain=['counter-factual'], range=['blue']), legend=None), 
                    strokeDash=alt.StrokeDash('which:N', scale=alt.Scale(domain=['counter-factual'], range=[[2,2]]), legend=alt.Legend(title='' ))                              
                    )
diff = round(treatment_post - treat_counter_fact, 1)
dftext = pd.DataFrame({
 'text': [str(diff) + ' more mean rides per minute', 'in post-treatment period'],
 'tx' : [str(df_did['timestamp'].max()), str(df_did['timestamp'].max())],
 'ty': [450, 400]})


text = alt.Chart(dftext).mark_text(color='black', align='left', dx=-60, dy=-70, fontSize=15).encode(
 x='tx:T',
 y=alt.Y('ty:Q'
 ),
 text='text'
)


# https://github.com/altair-viz/altair/issues/1694
(rides + boundary_line + did_points + did_lines + count_fact_line + text).resolve_scale(color='independent').display()





alt.LayerChart(...)

In [ ]:
df_did

,b_id,b,which,timestamp,rides_per_min
0,0,2010-01-17 11:51:00,treatment,2010-01-17 10:51:00,279.625000
1,0,2010-01-17 11:51:00,treatment,2010-01-17 12:51:00,470.041322
2,0,2010-01-17 11:51:00,counter-factual,2010-01-17 10:51:00,279.625000
3,0,2010-01-17 11:51:00,control,2010-01-17 10:51:00,311.520833
4,0,2010-01-17 11:51:00,control,2010-01-17 12:51:00,415.351240
5,0,2010-01-17 11:51:00,counter-factual,2010-01-17 12:51:00,383.455406


In [ ]:
df_did[df_did['which']!='counter-factual']

,b_id,b,which,timestamp,rides_per_min
0,0,2010-01-17 11:51:00,treatment,2010-01-17 10:51:00,279.625000
1,0,2010-01-17 11:51:00,treatment,2010-01-17 12:51:00,470.041322
3,0,2010-01-17 11:51:00,control,2010-01-17 10:51:00,311.520833
4,0,2010-01-17 11:51:00,control,2010-01-17 12:51:00,415.351240


In [ ]:
df_did['which_week_dummy'] = (df_did['which'] == 'treatment').astype(int)
df_did['afternoon_dummy'] = (df_did['timestamp'] >  df_did['b']).astype(int)

In [ ]:
df_ols = df_did[df_did['which']!='counter-factual']

#Import statements
import statsmodels.formula.api as smf
#Results of base model
reg = smf.ols(formula = 'rides_per_min ~ which_week_dummy*afternoon_dummy', data = df_ols).fit()

ols_robust2_1 = reg.get_robustcov_results(cov_type= 'HC1') 
ols_robust2_1.summary()

/root/venv/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1650: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/root/venv/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1886: RuntimeWarning: divide by zero encountered in double_scalars
  self.het_scale = self.nobs/(self.df_resid)*(self.wresid**2)
/root/venv/lib/python3.7/site-packages/statsmodels/stats/stattools.py:75: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  "samples were given." % int(n), ValueWarning)
/root/venv/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1728: RuntimeWarning: divide by zero encountered in true_divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/root/venv/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1729: RuntimeWarning: invalid value encountered in double_scalars
  * (1 - self.rsquared))


ValueError: r_matrix performs f_test for using dimensions that are asymptotically non-normal

In [ ]:
df_3wfl

,b_id,b_ts,pickup_datetime_1min,dist_bin,rides_per_minute,which_week,sequence,pickup_datetime_same_time,which_week_dummy,before_after_rain_dummy,within_two_hours_dummy
0,0,2010-01-17 11:51:00,2010-01-10 05:51:00,8.0,10,Week before,1,2010-01-17 05:51:00,0,0,0
1,0,2010-01-17 11:51:00,2010-01-10 05:51:00,16.0,3,Week before,2,2010-01-17 05:51:00,0,0,0
2,0,2010-01-17 11:51:00,2010-01-10 05:51:00,2.0,12,Week before,3,2010-01-17 05:51:00,0,0,0
3,0,2010-01-17 11:51:00,2010-01-10 05:51:00,1.0,6,Week before,4,2010-01-17 05:51:00,0,0,0
4,0,2010-01-17 11:51:00,2010-01-10 05:52:00,2.0,8,Week before,5,2010-01-17 05:52:00,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1455504,79,2010-12-26 09:51:00,2010-12-26 15:51:00,1.0,10,Current week,5813,2010-12-26 15:51:00,1,1,0
1455505,79,2010-12-26 09:51:00,2010-12-26 15:51:00,2.0,11,Current week,5814,2010-12-26 15:51:00,1,1,0
1455506,79,2010-12-26 09:51:00,2010-12-26 15:51:00,4.0,13,Current week,5815,2010-12-26 15:51:00,1,1,0
1455507,79,2010-12-26 09:51:00,2010-12-26 15:51:00,0.5,3,Current week,5816,2010-12-26 15:51:00,1,1,0


In [ ]:
df_3wfl[(df_3wfl['within_two_hours_dummy']==1) & (df_3wfl['b_id']==0)]

,b_id,b_ts,pickup_datetime_1min,dist_bin,rides_per_minute,which_week,sequence,pickup_datetime_same_time,which_week_dummy,before_after_rain_dummy,within_two_hours_dummy
1988,0,2010-01-17 11:51:00,2010-01-10 09:51:00,0.00,1,Week before,1989,2010-01-17 09:51:00,0,0,1
1989,0,2010-01-17 11:51:00,2010-01-10 09:51:00,1.00,60,Week before,1990,2010-01-17 09:51:00,0,0,1
1990,0,2010-01-17 11:51:00,2010-01-10 09:51:00,2.00,84,Week before,1991,2010-01-17 09:51:00,0,0,1
1991,0,2010-01-17 11:51:00,2010-01-10 09:51:00,4.00,55,Week before,1992,2010-01-17 09:51:00,0,0,1
1992,0,2010-01-17 11:51:00,2010-01-10 09:51:00,0.50,12,Week before,1993,2010-01-17 09:51:00,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
16362,0,2010-01-17 11:51:00,2010-01-24 13:51:00,8.00,33,Week after,4030,2010-01-17 13:51:00,0,1,1
16363,0,2010-01-17 11:51:00,2010-01-24 13:51:00,0.50,35,Week after,4031,2010-01-17 13:51:00,0,1,1
16364,0,2010-01-17 11:51:00,2010-01-24 13:51:00,16.00,15,Week after,4032,2010-01-17 13:51:00,0,1,1
16365,0,2010-01-17 11:51:00,2010-01-24 13:51:00,32.00,5,Week after,4033,2010-01-17 13:51:00,0,1,1


In [ ]:
#df_3wfl[(df_3wfl['within_two_hours_dummy']==1]) & (df_3wfl.b_id.isin(great))]


#df_3wfl[df_3wfl['within_two_hours_dummy']==1].b_id.isin(great)


df_greats = df_3wfl[df_3wfl.b_id.isin(great)]

df_greats = df_greats[df_greats['within_two_hours_dummy']==1]

In [ ]:
df_ols

,b_id,b,which,timestamp,rides_per_min,which_week_dummy,afternoon_dummy
0,0,2010-01-17 11:51:00,treatment,2010-01-17 10:51:00,279.625000,1,0
1,0,2010-01-17 11:51:00,treatment,2010-01-17 12:51:00,470.041322,1,1
3,0,2010-01-17 11:51:00,control,2010-01-17 10:51:00,311.520833,0,0
4,0,2010-01-17 11:51:00,control,2010-01-17 12:51:00,415.351240,0,1


In [ ]:
df_ols = df_3wfl[(df_3wfl['within_two_hours_dummy']==1) & (df_3wfl['b_id']==0)]
#df_ols = df_greats

reg = smf.ols(formula = 'rides_per_minute ~ which_week_dummy + before_after_rain_dummy + which_week_dummy:before_after_rain_dummy', data = df_ols).fit()

ols_robust2_1 = reg.get_robustcov_results(cov_type= 'HC1') 
ols_robust2_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       rides_per_minute   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     34.47
Date:                Wed, 19 May 2021   Prob (F-statistic):           4.38e-22
Time:                        17:18:08   Log-Likelihood:                -32625.
No. Observations:                6215   AIC:                         6.526e+04
Df Residuals:                    6211   BIC:                         6.528e+04
Df Model:                           3                                         
Covariance Type:                  HC1                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                   41.3592      0.966     42.796      0.000      39.465      43.254
which_week_dummy                            -7.9060      1.460     -5.414      0.000     -10.769      -5.043
before_after_rain_dummy                      1.7467      1.402      1.246      0.213      -1.002       4.495
which_week_dummy:before_after_rain_dummy    18.5985      2.519      7.382      0.000      13.660      23.537
==============================================================================
Omnibus:                      743.243   Durbin-Watson:                   1.021
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1044.229
Skew:                           1.003   Prob(JB):                    1.77e-227
Kurtosis:                       2.902   Cond. No.                         6.34
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [ ]:
diff_df = df_3wfl.copy() # df for diff in diff

In [ ]:
diff_df.head()

,b_id,b_ts,pickup_datetime_1min,dist_bin,rides_per_minute,which_week,sequence,pickup_datetime_same_time,which_week_dummy,before_after_rain_dummy,within_two_hours_dummy
0,0,2010-01-17 11:51:00,2010-01-10 05:51:00,8.0,10,Week before,1,2010-01-17 05:51:00,0,0,0
1,0,2010-01-17 11:51:00,2010-01-10 05:51:00,16.0,3,Week before,2,2010-01-17 05:51:00,0,0,0
2,0,2010-01-17 11:51:00,2010-01-10 05:51:00,2.0,12,Week before,3,2010-01-17 05:51:00,0,0,0
3,0,2010-01-17 11:51:00,2010-01-10 05:51:00,1.0,6,Week before,4,2010-01-17 05:51:00,0,0,0
4,0,2010-01-17 11:51:00,2010-01-10 05:52:00,2.0,8,Week before,5,2010-01-17 05:52:00,0,0,0


In [ ]:
#diff_df.iloc[0]['pickup_datetime_1min'] -  pd.Timedelta(hours=3) 
diff_df.iloc[0]['b_ts'] -  pd.Timedelta(hours=3) 

Timestamp('2010-01-17 08:51:00')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e5c38169-dd81-4fdb-9628-d1699354129b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>